In [81]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import os
import requests
from dotenv import load_dotenv
import base64
import json
import webbrowser
import urllib
import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
## https://developer.spotify.com/documentation/web-api/reference/get-users-saved-tracks

In [49]:
load_dotenv()
REDIRECT_URI = "http://localhost"
CLIENT_ID = os.getenv("client_id")
CLIENT_SECRET = os.getenv("client_secret")
SCOPE = 'user-library-read'

## Request User Authorization

In [50]:
# Create the authorization URL
params = {
    'response_type': 'code',
    'client_id': CLIENT_ID,
    'scope': SCOPE,
    'redirect_uri': REDIRECT_URI,
}
url = f"https://accounts.spotify.com/authorize?{urllib.parse.urlencode(params)}"

# Open the authorization URL in the default web browser
webbrowser.open(url)

True

In [51]:
CODE = "AQDeWZH2iptg8sDi07Ve9P-8RFGt1WKvIfcXGP0eBzcjU4zlGRu34V6FUBXeVNaIMTKGZphXctlcgUwP0qvH_fixYEChgFlKTGN6fkGNAcQ7Htk15hJWmtbW3aKrDPDQ1VjXiljTFcA9KtVS9Kkc4EaU22F-uodTkm0gD9fZ_Ae2vTbd_8Ab"

## Get Token Info
- Lasts only 1 hour

In [52]:
def get_token_info():
    auth_header =base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode('utf-8')).decode('utf-8')
    url ="https://accounts.spotify.com/api/token"
    headers={
            "Authorization": f"Basic {auth_header}",
            "Content-Type": "application/x-www-form-urlencoded",}
    data={
            "grant_type": "authorization_code",
            "code" : CODE,
            "redirect_uri": REDIRECT_URI}
    result=requests.post(url, headers=headers, data=data)
    json_results=json.loads(result.content)
    return json_results

token_details=get_token_info()

In [53]:
token_details

{'access_token': 'BQBK8RlJt9qWUl0YL29GJhQXcLSYaiz67clAgLA0J2IKiy2ylXb-eMi52Mgn0I9eeIQzjKJlT1YCwNb-KyVDxPT8ShU5s_szLNkIpjRnYM5rMS4JTYWqPBt0nfcaoKFtQesB_QpgdXXPsNNKngHPn5zipnd8jEInDR5tQzUogE1cGjlCg9kTqqRurgTtb6U',
 'token_type': 'Bearer',
 'expires_in': 3600,
 'refresh_token': 'AQApIGFhTZUUgoyz9LwfJ9VCAlvKoyXXkG8PxzyT15qgAMPX2PtgMaJ22_mlyXQOf3sK7zPLlGZTxEBA5zUiQCga93sJiaUbKfPuBu_rYa4yBJh0KqVfYpiRry3ZOZlEkyQ',
 'scope': 'user-library-read'}

In [54]:
token = token_details["access_token"]
refresh_token = token_details["refresh_token"]
scope = token_details["scope"]

## Get Auth Header

In [55]:
def get_auth_header(token):
    return f"Bearer " + token

## Post Request
- Request using token for API endpoints

In [56]:
SPOTIFY_GET_CURRENT_USER_SAVED_TRACKS_URL = "https://api.spotify.com/v1/me/tracks?limit=50&offset=0"

In [57]:
def get_current_user_tracks(url):
    response = requests.get(url=url, headers={"Authorization" : f"{get_auth_header(token)}"})
    resp_json = response.json()
    return resp_json

In [59]:
def loop_through():
    first_spotify_url = "https://api.spotify.com/v1/me/tracks?limit=50&offset=0"
    results = get_current_user_tracks(first_spotify_url)
    full_list = []
    current_url = results["href"]
    while results["next"] is not None:
    # for i in range(4):
        results = get_current_user_tracks(current_url)
        full_list.append(results)
        spotify_url_next = results["next"]
        current_url = spotify_url_next
    return full_list
            

In [60]:
saved_tracks_data = loop_through()

## Results

In [62]:
saved_tracks_parsed={}
for index_i, i in enumerate(saved_tracks_data):  
    for index_j, j in enumerate(saved_tracks_data[index_i]["items"]):
        saved_tracks_parsed[j["track"]["uri"]] = {
              "added_to_playlist_time":j["added_at"]
            , "artist_name":j["track"]["album"]["artists"][0]["name"]
            , "unique_artist_id": j["track"]["album"]["artists"][0]["uri"]
            , "release_date": j["track"]["album"]["release_date"]
            , "song_name": j["track"]["name"]
            , "song_length_seconds": str(int(j["track"]["duration_ms"])/1000)
            , "song_explicit": str(j["track"]["explicit"])
            , "song_linkId": j["track"]["external_urls"]["spotify"]
            , "song_popularity": str(j["track"]["popularity"])
            , "album_name": j["track"]["album"]["name"]
            , "spotify_uri_album": j["track"]["album"]["uri"]
            , "spotify_track_id": j["track"]["id"]  
                                            }

In [65]:
len(saved_tracks_parsed)

4489

## Get song genres from Get Track API Endpoint
### Seems as if most songs do not have a genre object/attribute associated with them (shelving this for now)

In [66]:
# for i in full_list_parsed[0:10]:
#     print(i[0])

# full_list_parsed[0]["0.0"][-1].split(":")[-1]
# list(full_list_parsed[0].keys())

In [67]:
def get_track_genres(url):
    response = requests.get(url=url, headers={"Authorization" : f"{get_auth_header(token)}"})
    resp_json = response.json()
    return resp_json


get_track_genres("https://api.spotify.com/v1/tracks?ids=6POZ7ocsOQQvJrfArz6KOD")
# for track_id in full_list_parsed:
#     get_track_genre_url= f"https://api.spotify.com/v1/tracks/{track_id}"
#     get_track_genres(get_track_genre_url)

{'tracks': [{'album': {'album_type': 'single',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/757FXqX0Osk2pqtgv4E5v4'},
      'href': 'https://api.spotify.com/v1/artists/757FXqX0Osk2pqtgv4E5v4',
      'id': '757FXqX0Osk2pqtgv4E5v4',
      'name': 'NURKO',
      'type': 'artist',
      'uri': 'spotify:artist:757FXqX0Osk2pqtgv4E5v4'},
     {'external_urls': {'spotify': 'https://open.spotify.com/artist/0p3tzEAt0XWrBqbrwBoN1I'},
      'href': 'https://api.spotify.com/v1/artists/0p3tzEAt0XWrBqbrwBoN1I',
      'id': '0p3tzEAt0XWrBqbrwBoN1I',
      'name': 'Kyle Hume',
      'type': 'artist',
      'uri': 'spotify:artist:0p3tzEAt0XWrBqbrwBoN1I'}],
    'available_markets': ['AD',
     'AE',
     'AG',
     'AL',
     'AM',
     'AO',
     'AR',
     'AT',
     'AU',
     'AZ',
     'BA',
     'BB',
     'BD',
     'BE',
     'BF',
     'BG',
     'BH',
     'BI',
     'BJ',
     'BN',
     'BO',
     'BR',
     'BS',
     'BT',
     'BW',
     'BZ',
     'CA',
 

## Call Track Auto Features API endpoint
- https://developer.spotify.com/documentation/web-api/reference/get-several-audio-features

In [68]:
def get_tracks_audio_analysis(url):
    response = requests.get(url=url, headers={"Authorization" : f"{get_auth_header(token)}"})
    resp_json = response.json()
    return resp_json

### Creating string for audio track inputs
- Spotify limits the inputs to 100 track ids at a time

In [69]:
listofids=[]
for index, i in enumerate(list(saved_tracks_parsed.values())):
    listofids.append(i["spotify_track_id"])

In [71]:
len(listofids)/100

44.89

## Breaking up Id Lists
- Track Audio Analysis only takes in 100 ids at a time

In [72]:
def list_chunks(l,n):
    for i in range(0,len(l), n):
        yield l[i:i + n]

n = 100

chunksoftrackids=list(list_chunks(listofids,n))

In [74]:
len(chunksoftrackids)

45

In [75]:
string_urls=[]
for index,chunk in enumerate(chunksoftrackids):
    string_urls.append(','.join(chunk))

## Getting Track Audio Features

In [76]:
track_audio_features={}
for index,i in enumerate(string_urls):
    audiofeatures= get_tracks_audio_analysis(f'https://api.spotify.com/v1/audio-features?ids={string_urls[index]}')
    for j in audiofeatures["audio_features"]:
        track_audio_features[j['uri']] ={
                "danceability":j["danceability"]
                , "energy":j["energy"]
                , "key":j["key"]
                , "loudness":j["loudness"]
                , "mode":j["mode"]
                , "speechiness":j["speechiness"]
                , "acousticness":j["acousticness"]
                , "instrumentalness":j["instrumentalness"]
                , "liveness":j["liveness"]
                , "valence":j["valence"]
                , "time_signature":j["time_signature"]
                , "tempo":j["tempo"]
                                        }

In [77]:
print(track_audio_features['spotify:track:4p16E9c9Ig6xFMGS3Y82mT'])
print(saved_tracks_parsed['spotify:track:4p16E9c9Ig6xFMGS3Y82mT'])

{'danceability': 0.494, 'energy': 0.917, 'key': 7, 'loudness': -3.302, 'mode': 0, 'speechiness': 0.144, 'acousticness': 0.0381, 'instrumentalness': 0, 'liveness': 0.0943, 'valence': 0.178, 'time_signature': 4, 'tempo': 145.847}
{'added_to_playlist_time': '2023-06-09T15:26:19Z', 'artist_name': 'RL Grime', 'unique_artist_id': 'spotify:artist:5eIbEEQnDM8yuDVB0bimSP', 'release_date': '2023-06-09', 'song_name': 'Pour Your Heart Out (feat. 070 Shake)', 'song_length_seconds': '223.561', 'song_explicit': 'False', 'song_linkId': 'https://open.spotify.com/track/4p16E9c9Ig6xFMGS3Y82mT', 'song_popularity': '67', 'album_name': 'Pour Your Heart Out (feat. 070 Shake)', 'spotify_uri_album': 'spotify:album:3IqciDNeCpyycdfxs6AA6T', 'spotify_track_id': '4p16E9c9Ig6xFMGS3Y82mT'}


In [78]:
tracks_fulldetails={}
for k in saved_tracks_parsed:
    tracks_fulldetails[k]=[saved_tracks_parsed[k], track_audio_features[k]]

In [80]:
len(tracks_fulldetails)

4489

In [84]:
tracks_fulldetails['spotify:track:4p16E9c9Ig6xFMGS3Y82mT'][1]

{'danceability': 0.494,
 'energy': 0.917,
 'key': 7,
 'loudness': -3.302,
 'mode': 0,
 'speechiness': 0.144,
 'acousticness': 0.0381,
 'instrumentalness': 0,
 'liveness': 0.0943,
 'valence': 0.178,
 'time_signature': 4,
 'tempo': 145.847}

In [90]:
pd.DataFrame.from_records(list(tracks_fulldetails.values())[0])

,added_to_playlist_time,artist_name,unique_artist_id,release_date,song_name,song_length_seconds,song_explicit,song_linkId,song_popularity,album_name,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,time_signature,tempo
0,2023-06-09T15:26:19Z,RL Grime,spotify:artist:5eIbEEQnDM8yuDVB0bimSP,2023-06-09,Pour Your Heart Out (feat. 070 Shake),223.561,False,https://open.spotify.com/track/4p16E9c9Ig6xFMG...,67,Pour Your Heart Out (feat. 070 Shake),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,-3.302,0.0,0.144,0.0381,0.0,0.0943,0.178,4.0,145.847
